# Elektronische Bauelemente: MOS transistor

Dies ist eine interaktive Oberfläche zur Erkundung verschiedener Zusammenhänge des MOS transistor auf Basis der Vorlesung

In [1]:
import numpy as np
import math

from bqplot import (
    LogScale, LinearScale, OrdinalColorScale, ColorAxis, ColorScale,
    Axis, Scatter, Lines, CATEGORY10, Label, Figure, Tooltip
)

from ipywidgets import (
    HBox, VBox, FloatText, FloatRangeSlider, IntSlider, FloatSlider, 
    Play, jslink, FloatLogSlider, interactive_output, HTML, HTMLMath, Layout, Checkbox, Label, Text
)

from bqplot import pyplot as plt
from bqplot import topo_load
from bqplot.interacts import panzoom

from IPython.display import Math

# global constants
eps0        = 8.854e-12     # [As/(Vm)] - permittivity of free space
q           = 1.602e-19     # [As]      - electronic charge
kb          = 1.38e-23      # [J/K]     - Boltzmann's constant

# global fixed parameters
T           = 300           # [K]       - temperature
UT          = kb*T/q        # [V]       - thermal voltage
epsr_si     = 11.7          # [ ]       - relative permittivity of Si
epsr_ox     = 3.73          # [ ]       - relative permittivity of SiO2
ni          = 1.07e10       # [cm^-3]    - intrinsic carrier concentration in Si (at 300 K)
Lch         = 0.5e-6        # [m]       - channel length
mu_n        = 0.1           # [m^2/(Vs)]- mobility of electrons in Si
Uth0        = 0.7           # [V]       - threshold voltage
UA          = 50            # [V]       - early voltage

# Bias point initialization
Ugs_sweep   = np.linspace(0,2.5,101)     # [V]   - gate-source voltage
Uds_sweep   = np.linspace(0,5,101)      # [V]   - drain-source voltage
Usb_sweep   = np.linspace(0,5,101)      # [V]   - source-substrate voltage

### Widget definitions
Uth0_wid = Label(value=r'Threshold voltage \(U_{\rm{th0}}\)'+' = '+str(Uth0)+' V') # threshold voltage - Uth0
Lch_wid = Label(value=r'Channel length \(L\)'+' = '+str(Lch*1e6)+' µm') # Channel length - L
mu_n_wid = Label(value=r'Electron mobility \(\mu_{\rm{n}}\)'+' = '+str(mu_n*1e4)+r' \(\rm{cm^{2}/(Vs)}\)') # Electron mobility 
UA_wid = Label(value=r'Early voltage \(U_{\rm{A}}\)'+' = '+str(UA)+' V') # Early voltage
ni_wid = Label(value=r'Intrinsic concentration  \(n_{\rm{i}}\)'+' = '+str(ni/1e10)+r' \(\rm{\times 10^{10} cm^{-3}}\)') # Intrinsic concentration
temp_wid = Label(value=r'Temperature \(T\)'+' = '+str(T)+' K') # Temperature

### Equations
IDeqn_wid = HTMLMath(
    value=r"Drain current: $$I_{\mathrm{D,lin}} = k'_{n}\left[(U_{GS}-U_{th})U_{DS} - (1+a_{th})\frac{U_{DS}^2}{2}\right]$$", # \text{for $U_{GS}<U_{th},U_{DS}\le U_{DS,sat}$}$$", # \\ I_{\mathrm{D}} = k'_{n}\left[\frac{1-k_{clm}}{2(1+a_{th})}(U_{GS}-U_{th})^2 + \frac{k_{clm}}{2}(U_{GS}-U_{th})U_{DS})\right] \text{for $U_{GS}<U_{th},U_{DS}>U_{DS,sat}$}\\ I_{\mathrm{D}} = 0 \text{ for $U_{GS}<U_{th}$}$$",
#     description='Drain current',
)


### Variables and their slider definitions
# Width (W)
W_slider = FloatSlider(
    value=5,
    min=1,
    max=10,
    step=0.5,
    description=r'\(W/\rm{\mu m}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    # style=style
)

# Oxide thickness (d_ox)
dox_slider = FloatSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description=r'\(d_{\rm{ox}}/\rm{nm}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    # style=style
)

# Acceptor concentration (N_A)
NA_slider = FloatLogSlider(
    value=5e16,
    base=10,
    min=15, # max exponent of base
    max=17, # min exponent of base
    step=1e-1, # exponent step
    description=r'\(N_{\rm{A}}^{-}/\rm{cm^{-3}}\)',
    continuous_update=False,
    # style=style
)

# Terminal voltage and its slider definitions
# Gate-source voltage (U_GS) for Output characteristics
Ugs_slider = FloatRangeSlider(
    value=[1, 2.5],
    min=0.5,
    max=5.0,
    step=0.5,
    description=r'\(U_{\rm{GS}}/\rm{V}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    # style=style
)
# Drain-source voltage (U_DS) for transfer characteristics
Uds_slider = FloatRangeSlider(
    value=[0.1, 3],
    min=0,
    max=5.0,
    step=0.1,
    description=r'\(U_{\rm{DS}}/\rm{V}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    # style=style
)
# source-substrate voltage (U_SB) for plotting Uth vs Usb
Usb_slider = FloatSlider(
    value=0,
    min=np.min(Usb_sweep),
    max=np.max(Usb_sweep),
    step=0.5,
    description=r'\(U_{\rm{SB}}/\rm{V}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    # style=style
)

# function definitions
def calc_Cox(d_ox):
    """Function to oxide capacitance C_ox
    Input
    ------
    d_ox    : float
        oxide thickness.
    Output
    ---------
    C_ox    : float
        oxide capacitance per unit area.
    """
    d_ox_m = d_ox
    C_ox = eps0*epsr_ox/d_ox_m
    return C_ox

def calc_kn(W,L,mu_n):
    """Function to calculate the drain current prefactor.
    Input
    ------
    W       : float 
        gate width.
    L       : float 
        channel length.
    mu_n    : float
        electron mobility.
    Output
    ---------
    km      : float
        drain current prefactor (per unit area).
    """
    C_ox = calc_Cox(dox_slider.value*1e-9)
    
    kn = W/L*mu_n*C_ox
    return kn

def calc_gamma():
    """Function to calculate gamma, i.e. body effect coefficient (see 2.5.19 in [1]).
    Input
    ------
    None
    Output
    ---------
    gamma   : float
        body effect coefficient.
    """
    C_ox = calc_Cox(dox_slider.value*1e-9)
    
    gamma = (1/C_ox)*np.sqrt(2*eps0*epsr_si*q*NA_slider.value*1e6)
    return gamma

def calc_phib():
    """Function to calculate phi_b, i.e. the upper limit of weak inversion (see 2.5.25 in [1]).
    Input
    ------
    None
    Output
    ---------
    phi_b   : float
        the upper limit of weak inversion.
    """
    phi_f = UT*np.log(NA_slider.value/ni)

    phi_b = 2*phi_f
    return phi_b

def calc_ath(Usb):
    """Function to calculate a_th, i.e. the slope of extrapolated threshold voltage Uth vs Usb (see 4.4.33b in [1]).
    Input
    ------
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    a_th    : float
        slope of extrapolated threshold voltage Uth vs Usb.
    """
    gamma   = calc_gamma()
    phi_b   = calc_phib()

    a_th = 0.5*gamma/np.sqrt(phi_b+Usb)
    return a_th

def calc_Uth(Uth0,Usb):
    """Function to calculate Uth, i.e. gate-source extrapolated threshold voltage (see 4.4.26b in [1]).
    Input
    ------
    Uth0     : float 
        threshold voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    Uth    : float
        gate-source extrapolated threshold voltage.
    """
    gamma   = calc_gamma()
    phi_b   = calc_phib()

    Uth = Uth0 + gamma*(np.sqrt(phi_b+Usb) - np.sqrt(phi_b))
    return Uth

def calc_ID(Ugs,Uds,Usb):
    """Function to calculate the drain current ID for single op point (see 4.4.30 in [1]).
    Input
    ------
    Ugs     : float 
        gate-source voltage.
    Uds     : float 
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID      : float
        Drain current.
    """

    kn      = calc_kn(W_slider.value*1e-6,Lch,mu_n)
    a_th    = calc_ath(Usb)
    Uth     = calc_Uth(Uth0,Usb)
    
    Uds_sat = (Ugs-Uth)/(1+a_th)
    k_clm   = Uds_sat/(UA+Uds_sat)

    if Ugs < Uth:
        ID = 0
    else:
        if Uds <= Uds_sat:
            ID = kn*((Ugs-Uth)*Uds - 0.5*(1+a_th)*Uds**2)
        else:
            ID = kn*((1-k_clm)/(2*(1+a_th))*(Ugs-Uth)**2 + 0.5*k_clm*(Ugs-Uth)*Uds)

    # print(ID)
    return ID

def calc_ID_gds(Ugs,Uds,Usb):
    """Function to calculate the output characteristics and output conductance
    Input
    ------
    Ugs     : list 
        gate-source voltage.
    Uds     : ndarray 
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID_out  : ndarray
        Drain current (output characteristics).
    gds     : ndarray
        Output conductance.
    """

    ID_out  = np.zeros((len(Ugs),len(Uds)))
    gds     = np.zeros((len(Ugs),len(Uds)))

    for nUgs in np.arange(np.size(Ugs)):
        for nUds in np.arange(np.size(Uds)):
            ID_out[nUgs,nUds] = calc_ID(Ugs[nUgs],Uds[nUds],Usb)
        gds[nUgs,:] = np.gradient(ID_out[nUgs,:])/np.gradient(Uds)

    return ID_out,gds

def calc_ID_gm(Ugs,Uds,Usb):
    """Function to calculate the transfer characteristics and transconductance
    Input
    ------
    Ugs     : ndarray  
        gate-source voltage.
    Uds     : list
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID_trans    : ndarray
        Drain current (transfer characteristics).
    gm          : ndarray
        transconductance.
    """

    ID_trans    = np.zeros((len(Uds),len(Ugs)))
    gm          = np.zeros((len(Uds),len(Ugs)))

    for nUds in np.arange(np.size(Uds)):
        for nUgs in np.arange(np.size(Ugs)):
            ID_trans[nUds,nUgs] = calc_ID(Ugs[nUgs],Uds[nUds],Usb)
        gm[nUds,:]  = np.gradient(ID_trans[nUds,:])/np.gradient(Ugs)
    
    return ID_trans,gm

# Figure definitions
### other definitions
col_out     = ColorScale(scheme='RdYiBu') # viridis
col_trans   = ColorScale(scheme='RdYiBu')

### Layout definitions
fig_layout = Layout(
    align_self='center',
    width='350px',
    height='275px',
)

figLarge_layout = Layout(
    align_self='center',
    width='680px',
    height='400px',
)

### scale definitions
## x-axis
sc_x_Usb        = LinearScale(min=0)
sc_x_Uds        = LinearScale(min=0)
sc_x_Ugs        = LinearScale(min=0)
## y-axis
sc_y_Uth        = LinearScale(min=0.5)
sc_y_IDout      = LinearScale(min=0)
sc_y_IDtrans    = LinearScale(min=0)
sc_y_gds        = LinearScale(min=0)
sc_y_gm         = LinearScale(min=0)

### Axis definitions
## x-axis
ax_x_Usb        = Axis(scale=sc_x_Usb, label='Usb/V', grid_lines='solid')
ax_x_Uds        = Axis(scale=sc_x_Uds, label='Uds/V', grid_lines='solid', num_ticks=6)
ax_x_Ugs        = Axis(scale=sc_x_Ugs, label='Ugs/V', grid_lines='solid', num_ticks=6)
## y-axis
ax_y_Uth        = Axis(scale=sc_y_Uth, orientation='vertical', label='Uth/V', label_offset='-50', num_ticks=5)
ax_y_IDout      = Axis(scale=sc_y_IDout, orientation='vertical', label='ID/mA', label_offset='-50', num_ticks=8)
ax_y_IDtrans    = Axis(scale=sc_y_IDtrans, orientation='vertical', label='ID/mA', label_offset='-50', num_ticks=8)
ax_y_gds        = Axis(scale=sc_y_gds, orientation='vertical', label='gds/mS', label_offset='-50', num_ticks=8)
ax_y_gm         = Axis(scale=sc_y_gm, orientation='vertical', label='gm/mA', label_offset='-50', num_ticks=8)


### Begin Uth vs Usb fig
Uth = calc_Uth(Uth0,Usb_sweep)
Usb_op  = 0

line_Uth = Lines(
    x             = Usb_sweep, 
    y             = Uth, 
    scales        = {'x': sc_x_Usb, 'y': sc_y_Uth},
    display_legend= False,
    labels        = ['Uth'],
)
index = np.argmin(np.abs(Usb_sweep-Usb_slider.value))
line_op_usb = Lines(
    x             = np.linspace(Usb_slider.value,Usb_slider.value,11), 
    y             = np.linspace(1e-20, Uth[index],11), 
    scales        = {'x': sc_x_Usb, 'y': sc_y_Uth},
    display_legend= False,
    colors        = ['MidnightBlue'],
)

fig_UthPlot     = Figure(marks=[line_Uth,line_op_usb], axes=[ax_x_Usb, ax_y_Uth], title='Uth(Usb)', layout=figLarge_layout)
# end Uth vs Usb fig

### Begin output characterisitics and output conductance fig
Ugs_op  = [1,1.5,2,2.5]
Usb_op  = 0

[ID_out,gds]  = calc_ID_gds(Ugs_op,Uds_sweep,Usb_op)

line_out = Lines(
        x               = Usb_sweep, 
        y               = ID_out*1e3,
        scales          = {'x': sc_x_Uds, 'y': sc_y_IDout, 'color': col_out},
        display_legend  = True,
        color           = Ugs_op,
        labels          = [str(i) for i in Ugs_op]
    )

line_gds = Lines(
        x               = Usb_sweep, 
        y               = gds*1e3,
        scales          = {'x': sc_x_Uds, 'y': sc_y_gds, 'color': col_out},
        display_legend  = True,
        color           = Ugs_op,
        labels          = [str(i) for i in Ugs_op]
    )

fig_out     = Figure(marks=[line_out], axes=[ax_x_Uds, ax_y_IDout], title='ID(Uds)', layout=fig_layout)
fig_gds     = Figure(marks=[line_gds], axes=[ax_x_Uds, ax_y_gds], title='gds(Uds)', layout=fig_layout)
# end output characterisitics and output conductance fig

### Begin transfer characterisitics and transconductance fig
sc_x_trans = LinearScale(min=0)
sc_y_trans = LinearScale(min=0)

Uds_op = [0.1,3]
Usb_op = 0

[ID_trans,gm]  = calc_ID_gm(Ugs_sweep,Uds_op,Usb_op)

line_trans = Lines(
        x               = Ugs_sweep, 
        y               = ID_trans*1e3,
        scales          = {'x': sc_x_Ugs, 'y': sc_y_IDtrans, 'color': col_trans},
        display_legend  = True,
        color           = [1,2,3,4],
        labels          = [str(i) for i in Uds_op]
    )

line_gm = Lines(
        x               = Ugs_sweep, 
        y               = gm*1e3,
        scales          = {'x': sc_x_Ugs, 'y': sc_y_gm, 'color': col_trans},
        display_legend  = True,
        color           = [1,2,3,4],
        labels          = [str(i) for i in Uds_op]
        )

fig_trans     = Figure(marks=[line_trans], axes=[ax_x_Ugs, ax_y_IDtrans], title='ID(Ugs)', legend_location='top-left', layout=fig_layout)
fig_gm     = Figure(marks=[line_gm], axes=[ax_x_Ugs, ax_y_gm], title='gm (Ugs)', legend_location='top-left', layout=fig_layout)
# end transfer characterisitics and transconductance fig

def update_uthPlot(*args,**kwargs):
    """Function to update Uth vs Usb plot.
    """
    Usb_op = Usb_slider.value
    Uth = calc_Uth(Uth0,Usb_sweep)
    line_Uth.y = Uth

    index           = np.argmin(np.abs(Usb_sweep-Usb_op))
    line_op_usb.x   = np.linspace(Usb_op,Usb_op,11)
    line_op_usb.y   = np.linspace(1e-20, Uth[index],11)


def update_outTransPlot(*args,**kwargs):
    """Function to update transfer characteristics and transconductance plots.
    """
    update_outPlot()
    update_transPlot()

def update_outPlot(*args,**kwargs):
    """Function to update output characteristics and output conductance plots.
    """
    # get new op points from slider
    [Ugs_op_min,Ugs_op_max] = Ugs_slider.value
    nOp = (Ugs_op_max-Ugs_op_min)/0.5 + 1
    Ugs_op = np.linspace(Ugs_op_min,Ugs_op_max,int(nOp))
    Usb_op = Usb_slider.value

    # calculate ID and gds for new op points
    [ID_out,gds]    = calc_ID_gds(Ugs_op,Uds_sweep,Usb_op)

    # update lines
    line_out.y      = ID_out*1e3
    line_gds.y      = gds*1e3

    # update labels
    line_out.labels     = [str(i) for i in Ugs_op]
    line_gds.labels     = [str(i) for i in Ugs_op]

def update_transPlot(*args,**kwargs):
    """Function to update transfer characteristics and transconductance plots.
    """
    # get new op points from slider
    [Uds_op_min,Uds_op_max] = Uds_slider.value
    Uds_op = [Uds_op_min,Uds_op_max]
    Usb_op = Usb_slider.value

    # calculate ID and gm for new op points
    [ID_trans,gm]   = calc_ID_gm(Ugs_sweep,Uds_op,Usb_op)

    # update lines
    line_trans.y    = ID_trans*1e3
    line_gm.y       = gm*1e3
    # line_trans.color = Uds_op
    # line_gm.color = Uds_op

    # update labels
    line_trans.labels   = [str(i) for i in Uds_op]
    line_gm.labels      = [str(i) for i in Uds_op]

def update_all(*args,**kwargs):
    """Function to update all the plots.
    """
    update_uthPlot()
    update_outTransPlot()

# Figure update
dox_slider.observe(update_all)
NA_slider.observe(update_all)
Usb_slider.observe(update_all)

W_slider.observe(update_outTransPlot)

Ugs_slider.observe(update_outPlot)
Uds_slider.observe(update_transPlot)


# Layout definition
fixedPara_layout = Layout(
    flex_flow       = 'column',
    align_items   = 'stretch',
    width         = '265px'
)
parameter_layout = Layout(
    align_items   = 'baseline',
    width         = '100%',
    flex_grow       = 1
)

largeVBox_layout = Layout(
    align_items     = 'center',
    width         = '73%',
)

smallVBox_layout = Layout(
    align_content     = 'flex-end',
    width         = '27%',
)

center_layout = Layout(
    display         = 'flex',
    flex_flow       = 'column',
    align_items     = 'center',
    width           = '100%',
    flex_grow       = 1,
    justify_content = 'space-around'
)

final_layout = HBox([
        VBox([
                #HBox([
                #    HBox([Label('Equations')]),
                #    HBox([IDeqn_wid])
                #], layout=fixedPara_layout),
                HBox([
                    HBox([Label('Variable parameters')]),
                    HBox([W_slider]), 
                    HBox([dox_slider]),
                    HBox([NA_slider]),
                    HBox([Usb_slider]),
                ], layout=fixedPara_layout),
                HBox([
                    HBox([Label('Fixed parameters')]),
                    HBox([Uth0_wid]),
                    HBox([Lch_wid]),
                    HBox([mu_n_wid]),
                    HBox([ni_wid]),
                    HBox([UA_wid]),
                    HBox([temp_wid]),
                ], layout=fixedPara_layout),
                
            ], layout=smallVBox_layout),
        VBox([
                HBox([fig_UthPlot], layout=center_layout),
                VBox([
                    HBox([
                        HBox([fig_out,fig_gds]),
                        ]),
                    HBox([Ugs_slider], layout=center_layout)
                    ]),
                VBox([
                    HBox([
                        HBox([fig_trans,fig_gm]),
                        ]),
                    HBox([Uds_slider], layout=center_layout)
                    ]),
            ], layout=largeVBox_layout),
], layout=parameter_layout)


final_layout

# References

# [1] Y. Tsividis, Operation and modeling of the MOS transistor. New York, NY [u.a.]: McGraw-Hill, 1987.

